In [7]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from citipy import citipy
from datetime import datetime

# Import the requests library.
import requests

# Import the API key.
from config import weather_api_key

In [8]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key
print(url)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=03277455dc240e2a241a4dfed5409319


In [9]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs = np.random.uniform(low=-180.000, high=180.000, size=2000)
lat_lngs = zip(lats, lngs)
lat_lngs

In [10]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [11]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

740

In [12]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_weather_desc = city_weather["weather"][0]["description"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]

        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Country": city_country,        
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Current Description" : city_weather_desc,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

g Record 41 of Set 6 | valea larga
City not found. Skipping...
Processing Record 42 of Set 6 | puro
Processing Record 43 of Set 6 | grand river south east
City not found. Skipping...
Processing Record 44 of Set 6 | ambon
Processing Record 45 of Set 6 | talnakh
Processing Record 46 of Set 6 | east london
Processing Record 47 of Set 6 | roma
Processing Record 48 of Set 6 | redmond
Processing Record 49 of Set 6 | olavarria
Processing Record 50 of Set 6 | san matias
Processing Record 1 of Set 7 | high prairie
Processing Record 2 of Set 7 | abu dhabi
Processing Record 3 of Set 7 | cam pha
City not found. Skipping...
Processing Record 4 of Set 7 | bartlesville
Processing Record 5 of Set 7 | tasiilaq
Processing Record 6 of Set 7 | vaitape
Processing Record 7 of Set 7 | yenagoa
Processing Record 8 of Set 7 | jeremie
Processing Record 9 of Set 7 | bambous virieux
Processing Record 10 of Set 7 | gladstone
Processing Record 11 of Set 7 | new delhi
Processing Record 12 of Set 7 | sept-iles
Process

In [13]:
len(city_data)

677

In [14]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,Arraial Do Cabo,BR,-22.9661,-42.0278,91.40,57,0,20.71,clear sky,2021-01-28 20:07:50
1,Krasnoselkup,RU,65.7000,82.4667,-32.85,80,100,14.83,overcast clouds,2021-01-28 20:13:52
2,Punta Arenas,CL,-53.1500,-70.9167,48.20,57,75,23.02,broken clouds,2021-01-28 20:07:58
3,Novyy Urengoy,RU,66.0833,76.6333,-32.08,83,100,11.21,overcast clouds,2021-01-28 20:10:53
4,Nevers,FR,46.9167,3.3333,53.60,94,75,12.66,broken clouds,2021-01-28 20:13:53
5,Khatanga,RU,71.9667,102.5000,-40.27,75,0,3.51,clear sky,2021-01-28 20:12:35
6,Kavieng,PG,-2.5744,150.7967,82.83,76,59,9.37,broken clouds,2021-01-28 20:13:00
7,Kurumkan,RU,54.3000,110.3000,-20.56,78,100,4.00,overcast clouds,2021-01-28 20:13:53
8,Kapaa,US,22.0752,-159.3190,75.20,78,75,17.27,broken clouds,2021-01-28 20:12:02
9,Kijang,KR,35.2442,129.2139,24.80,33,0,14.97,clear sky,2021-01-28 20:13:54


In [15]:
# Create the output file (CSV).
output_data_file = "Weather_Database/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")